In [1]:
import numpy as np
from collections import Counter

In [2]:
corpus = ["the quick brown fox jumps over the lazy dog".split(' ')]
print(f'corpus example: {corpus}')

words = Counter()
for doc in corpus:
    for word in doc:
        words[word] += 1

vocab_size = len(words)
print(f'vocab size: {vocab_size}')

word_to_ix, ix_to_word = {}, {}
for ix, (word, _) in enumerate(words.most_common()):
    word_to_ix[word] = ix
    ix_to_word[ix] = word

corpus_preprocessed = list()
for document in corpus:
    corpus_preprocessed.append([word_to_ix[word] for word in document])
print(f'preprocessed: {corpus_preprocessed}')

corpus example: [['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']]
vocab size: 8
preprocessed: [[0, 1, 2, 3, 4, 5, 0, 6, 7]]


In [3]:
class Leaf:
    
    def __init__(self, value):
        self.path = list()
        self.value = value
    
    def __add__(self, value):
        self.path.append(value)
        return self

class Node:
    
    def __init__(self, v1, v2, h_size):
        self.left  = v1
        self.right = v2
        
        self.left  += 1
        self.right += -1
        
        self.value   = np.random.randn(h_size, 1)
        self.d_value = np.zeros_like(self.value)
    
    def __getitem__(self, index):
        node_len = len(self.leaves[index].path)
        
        nodes = []
        node = self
        for path in self.leaves[index].path:
            nodes.append((path, node))
            if path == 1:
                node = node.left
            else:
                node = node.right
                
        return nodes
    
    def __add__(self, value):
        self.left = self.left + value
        self.right = self.right + value
        return self
    
    @property
    def left(self):
        return self.__left
    
    @left.setter
    def left(self, value):
        self.__left = value
    
    @property
    def right(self):
        return self.__right
    
    @right.setter
    def right(self, value):
        self.__right = value
    
    @property
    def leaves(self):
        return self.__leaves
    
    @leaves.setter
    def leaves(self, leaves):
        self.__leaves = leaves
    
    def __repr__(self):
        return str(self.value)
    
def build_tree(leaves, h_size):
    ref_leaves = {leaf:Leaf(leaf) for leaf in leaves}
    leaves = list(ref_leaves.values())

    nodes = list()

    while True:
        if len(leaves) > 1:
            node = Node(leaves.pop(), leaves.pop(), h_size)
            nodes.append(node)
        elif len(nodes) > 1:
            node = Node(nodes.pop(), nodes.pop(), h_size)
            nodes.insert(0, node)
        else:
            node = Node(nodes.pop(), leaves.pop(), h_size)
            nodes.append(node)

        if len(leaves) == 0 and len(nodes) == 1:
            root = nodes[-1]
            root.leaves = ref_leaves
            return root

In [4]:
def get_data(corpus, window_size=2):
    for document in corpus:
        doc_len = len(document)
        for ix in range(doc_len):
            floor = max(0, ix-window_size)
            ceil = min(doc_len, ix+window_size+1)
            
            X = document[ix]
            
            y = list()
            for word in document[floor:ix]:
                y.append(word)
            for word in document[ix+1:ceil]:
                y.append(word)
            
            yield X, y

In [5]:
class EmbeddingModel(object):
    
    def __init__(self, vocab_size, h_size, learning_rate, seed=None):
        """
        parameters
        ----------
        
        vocabulary: list of unique words
        h_size    : size of the hidden units
        """
        if seed:
            np.random.seed(seed)
        
        self.eta = learning_rate
        
        self.w1   = np.random.randn(vocab_size, h_size)
        self.root = self.build_tree(range(vocab_size), h_size)
    
    def feed_forward(self, X, labels):
        h = self.w1[None, X]
        
        y_preds = list()
        for label in labels:
            y_pred = list()
            for sign, node in self.root[label]:
                y_pred.append((sign, self.sigmoid(np.dot(h, node.value))))
            y_preds.append(y_pred)

        return h, y_preds
    
    def __reset(self, node):
        if type(node.left) == Node:
            self.__reset(node.left)
        if type(node.right) == Node:
            self.__reset(node.right)
        node.d_value *= 0
    
    def __update(self, node):
        if type(node.left) == Node:
            self.__update(node.left)
        if type(node.right) == Node:
            self.__update(node.right)
        node.value -= self.eta * node.d_value
    
    def back_propagation(self, X, labels, y_preds, h):       
        dW1 = np.zeros((1, self.w1.shape[1]))
        
        self.__reset(self.root)
        
        for ix, (label, y_pred) in enumerate(zip(labels, y_preds)):
            for (_, y), (sign, node) in zip(y_pred, self.root[label]):
                error = y - 1 if sign == 1 else y
                
                dW1 += error * node.value.T

                node.d_value += error * h.T

        self.w1[None, X] -= self.eta * dW1
        self.__update(self.root)
    
    def build_tree(self, leaves, h_size):
        ref_leaves = {leaf:Leaf(leaf) for leaf in leaves}
        leaves = list(ref_leaves.values())

        nodes = list()

        while True:
            if len(leaves) > 1:
                node = Node(leaves.pop(), leaves.pop(), h_size)
                nodes.append(node)
            elif len(nodes) > 1:
                node = Node(nodes.pop(), nodes.pop(), h_size)
                nodes.insert(0, node)
            else:
                node = Node(nodes.pop(), leaves.pop(), h_size)
                nodes.append(node)

            if len(leaves) == 0 and len(nodes) == 1:
                root = nodes[-1]
                root.leaves = ref_leaves
                return root
    
    def sigmoid(self, x):
        return 1./(1.+np.exp(-x))
    
model = EmbeddingModel(vocab_size, 5, 1e-3, 42)

epochs = 15000

for epoch in range(epochs):
    loss = 0
    for X, labels in get_data(corpus_preprocessed):
        h, y_preds = model.feed_forward(X, labels)
        
        for y_pred in y_preds:
            for sign, value in y_pred:
                if sign == 1:
                    loss += -np.sum(np.log(value))
                else:
                    loss += -np.sum(np.log(1-value))

        model.back_propagation(X, labels, y_preds, h)

    if epoch % 1500 == 0:
        print(loss)

94.59578599323707
46.73028512617861
42.970845229988015
42.020892966453
41.660240599111624
41.481429923447735
41.37727575585884
41.308823047837606
41.2597761656506
41.2224404436544
